# Model 3 : Using Datetime Features to predict Energy Load

In [1]:
import os
import numpy as np 
import pandas as pd 
import plotly.graph_objs as go 
import plotly.io as pio
import plotly.express as px

import torch 
import torch.nn as nn 
import torch.optim as optim

pio.templates.default = "plotly_white"

from termcolor import colored

from datetime import datetime
import holidays

from utils import load_data

print(f"cwd : {os.getcwd()}")

cwd : /home/imantha/workspace/cryo-polygen/ts-forecasting/model3


## Load Data

In [2]:
path = os.path.join("..", "data", "load.xlsx")
df = load_data(path, last_date = "2022-05-15 23:00:00", endpoint=31)
df.rename(columns = {"y" : "value", "date" : "Datetime"}, inplace = True)
df.set_index("Datetime", inplace = True)
df = df[:3475]
df.head()

,value
Datetime,
2021-11-15 19:00:00,447
2021-11-15 20:00:00,435
2021-11-15 21:00:00,451
2021-11-15 22:00:00,442
2021-11-15 23:00:00,444


In [3]:
p = go.Figure()
p.add_trace(go.Scatter(
    x = df.index,
    y = df.value
))

## Lagged Observations as features

In [4]:
def generate_time_lags(df, n_lags):
    df_n = df.copy()
    for n in range(1, n_lags + 1):
        df_n[f"lag{n}"] = df_n["value"].shift(n)
    df_n = df_n.iloc[n_lags:]
    return df_n

df_generated = generate_time_lags(df, 24)
df_generated

,value,lag1,lag2,lag3,lag4,lag5,lag6,lag7,lag8,lag9,...,lag15,lag16,lag17,lag18,lag19,lag20,lag21,lag22,lag23,lag24
Datetime,,,,,,,,,,,,,,,,,,,,,
2021-11-16 19:00:00,488,484.0,439.0,436.0,436.0,446.0,498.0,472.0,482.0,510.0,...,459.0,480.0,476.0,447.0,446.0,444.0,442.0,451.0,435.0,447.0
2021-11-16 20:00:00,425,488.0,484.0,439.0,436.0,436.0,446.0,498.0,472.0,482.0,...,500.0,459.0,480.0,476.0,447.0,446.0,444.0,442.0,451.0,435.0
2021-11-16 21:00:00,432,425.0,488.0,484.0,439.0,436.0,436.0,446.0,498.0,472.0,...,468.0,500.0,459.0,480.0,476.0,447.0,446.0,444.0,442.0,451.0
2021-11-16 22:00:00,422,432.0,425.0,488.0,484.0,439.0,436.0,436.0,446.0,498.0,...,447.0,468.0,500.0,459.0,480.0,476.0,447.0,446.0,444.0,442.0
2021-11-16 23:00:00,417,422.0,432.0,425.0,488.0,484.0,439.0,436.0,436.0,446.0,...,468.0,447.0,468.0,500.0,459.0,480.0,476.0,447.0,446.0,444.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-09 09:00:00,508,530.0,538.0,582.0,491.0,475.0,504.0,515.0,492.0,500.0,...,524.0,532.0,533.0,518.0,565.0,637.0,628.0,674.0,752.0,748.0
2022-04-09 10:00:00,516,508.0,530.0,538.0,582.0,491.0,475.0,504.0,515.0,492.0,...,519.0,524.0,532.0,533.0,518.0,565.0,637.0,628.0,674.0,752.0
2022-04-09 11:00:00,503,516.0,508.0,530.0,538.0,582.0,491.0,475.0,504.0,515.0,...,505.0,519.0,524.0,532.0,533.0,518.0,565.0,637.0,628.0,674.0


## Generating Features from TimeStamps

In [5]:
df_features = (
    df
    .assign(hour = df.index.hour)
    .assign(day = df.index.day)
    .assign(month = df.index.month)
    .assign(day_of_week = df.index.dayofweek)
    .assign(week_of_year = df.index.week)
)

/tmp/ipykernel_21738/131758714.py:7: FutureWarning:

weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)



## One-hot encoding

In [6]:
df_features = pd.get_dummies(df_features, columns = ["month", "day", "day_of_week", "week_of_year"])

## Cyclic Features

In [7]:
def generate_cyclic_features(df, col_name, period, start_num = 0):
    kwargs = {
        f"sin_{col_name}" : lambda x: np.sin(2 * np.pi*(df[col_name] - start_num)/period),
        f"cos_{col_name}" : lambda x: np.cos(2 * np.pi*(df[col_name] - start_num)/period)
    }
    return df.assign(**kwargs).drop(columns = [col_name])

df_features = generate_cyclic_features(df_features, "hour", 24, 0)


In [8]:
df_features.head()

,value,month_1,month_2,month_3,month_4,month_11,month_12,day_1,day_2,day_3,...,week_of_year_14,week_of_year_46,week_of_year_47,week_of_year_48,week_of_year_49,week_of_year_50,week_of_year_51,week_of_year_52,sin_hour,cos_hour
Datetime,,,,,,,,,,,,,,,,,,,,,
2021-11-15 19:00:00,447,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,-0.965926,0.258819
2021-11-15 20:00:00,435,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,-0.866025,0.500000
2021-11-15 21:00:00,451,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,-0.707107,0.707107
2021-11-15 22:00:00,442,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,-0.500000,0.866025
2021-11-15 23:00:00,444,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,-0.258819,0.965926


## Holidays

In [9]:
us_holidays = holidays.Singapore()

def is_holiday(date):
    date = date.replace(hour = 0)
    return 1 if (date in us_holidays) else 0

def add_holiday_col(df, holidays):
    return df.assign(is_holiday = df.index.to_series().apply(is_holiday))

df_features = add_holiday_col(df_features, us_holidays)
df_features

,value,month_1,month_2,month_3,month_4,month_11,month_12,day_1,day_2,day_3,...,week_of_year_46,week_of_year_47,week_of_year_48,week_of_year_49,week_of_year_50,week_of_year_51,week_of_year_52,sin_hour,cos_hour,is_holiday
Datetime,,,,,,,,,,,,,,,,,,,,,
2021-11-15 19:00:00,447,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,-9.659258e-01,0.258819,0
2021-11-15 20:00:00,435,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,-8.660254e-01,0.500000,0
2021-11-15 21:00:00,451,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,-7.071068e-01,0.707107,0
2021-11-15 22:00:00,442,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,-5.000000e-01,0.866025,0
2021-11-15 23:00:00,444,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,-2.588190e-01,0.965926,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-09 09:00:00,508,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,7.071068e-01,-0.707107,0
2022-04-09 10:00:00,516,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,5.000000e-01,-0.866025,0
2022-04-09 11:00:00,503,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,2.588190e-01,-0.965926,0


In [10]:
df_generated.drop(columns = "value", inplace = True)
df_features = pd.merge(df_features,df_generated, left_index = True, right_index = True)
df_features

,value,month_1,month_2,month_3,month_4,month_11,month_12,day_1,day_2,day_3,...,lag15,lag16,lag17,lag18,lag19,lag20,lag21,lag22,lag23,lag24
Datetime,,,,,,,,,,,,,,,,,,,,,
2021-11-16 19:00:00,488,0,0,0,0,1,0,0,0,0,...,459.0,480.0,476.0,447.0,446.0,444.0,442.0,451.0,435.0,447.0
2021-11-16 20:00:00,425,0,0,0,0,1,0,0,0,0,...,500.0,459.0,480.0,476.0,447.0,446.0,444.0,442.0,451.0,435.0
2021-11-16 21:00:00,432,0,0,0,0,1,0,0,0,0,...,468.0,500.0,459.0,480.0,476.0,447.0,446.0,444.0,442.0,451.0
2021-11-16 22:00:00,422,0,0,0,0,1,0,0,0,0,...,447.0,468.0,500.0,459.0,480.0,476.0,447.0,446.0,444.0,442.0
2021-11-16 23:00:00,417,0,0,0,0,1,0,0,0,0,...,468.0,447.0,468.0,500.0,459.0,480.0,476.0,447.0,446.0,444.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-09 09:00:00,508,0,0,0,1,0,0,0,0,0,...,524.0,532.0,533.0,518.0,565.0,637.0,628.0,674.0,752.0,748.0
2022-04-09 10:00:00,516,0,0,0,1,0,0,0,0,0,...,519.0,524.0,532.0,533.0,518.0,565.0,637.0,628.0,674.0,752.0
2022-04-09 11:00:00,503,0,0,0,1,0,0,0,0,0,...,505.0,519.0,524.0,532.0,533.0,518.0,565.0,637.0,628.0,674.0


In [11]:
df_features.dtypes

value        int64
month_1      uint8
month_2      uint8
month_3      uint8
month_4      uint8
            ...   
lag20      float64
lag21      float64
lag22      float64
lag23      float64
lag24      float64
Length: 93, dtype: object

## Splitting the data into train, validation and test sets

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
def feature_label_split(df, target_col):
    y = df[[target_col]]
    X = df.drop(columns = [target_col])
    return X, y 

def train_val_test_split(df, target_col, test_ratio):
    val_ratio = test_ratio / (1 - test_ratio)
    X, y = feature_label_split(df, target_col)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, shuffle = False)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio, shuffle = False)
    return X_train, X_val, X_test, y_train, y_val, y_test 

X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(df_features, "value", 0.1)

print("shapes")
print(f"X_train : {X_train.shape}")
print(f"y_train : {y_train.shape}")
print(f"X_val : {X_val.shape}")
print(f"y_val : {y_val.shape}")
print(f"X_test : {X_test.shape}")
print(f"y_test : {y_test.shape}")

shapes
X_train : (2760, 92)
y_train : (2760, 1)
X_val : (345, 92)
y_val : (345, 1)
X_test : (346, 92)
y_test : (346, 1)


## Normalising Data

In [14]:
from sklearn.preprocessing import MinMaxScaler

In [15]:
normaliser = MinMaxScaler()
X_train_norm = normaliser.fit_transform(X_train)
X_val_norm = normaliser.fit_transform(X_val)
X_test_norm = normaliser.fit_transform(X_test)

y_train_norm = normaliser.fit_transform(y_train)
y_val_norm = normaliser.fit_transform(y_val)
y_test_norm = normaliser.fit_transform(y_test)

## Data Loader

In [16]:
from torch.utils.data import TensorDataset, DataLoader

In [17]:
batch_size = 64

train_features = torch.Tensor(X_train_norm)
train_targets = torch.Tensor(y_train_norm)
val_features = torch.Tensor(X_val_norm)
val_targets = torch.Tensor(y_val_norm)
test_features = torch.Tensor(X_test_norm)
test_targets = torch.Tensor(y_test_norm)

train_data = TensorDataset(train_features, train_targets)
val_data = TensorDataset(val_features, val_targets)
test_data = TensorDataset(test_features, test_targets)

train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = False, drop_last = True)
val_loader = DataLoader(val_data, batch_size = batch_size, shuffle = False, drop_last = True)
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = False, drop_last = True)


## Vanilla RNN

In [18]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, layer_size, output_size, dropout_prob):
        super(RNNModel, self).__init__()

        self.hidden_size = hidden_size
        self.layer_size = layer_size 
        # RNN Layer
        # (N,L,H_in), (numLayers,N, h_out) --> (N,L,H_out), (numlayers, n, hout)
        self.rnn = nn.RNN(input_size = input_size, hidden_size = hidden_size, batch_first = True)
        # Fully connected layers
        # (*, H_in) --> (*,H_out)
        self.fc = nn.Linear(in_features = hidden_size, out_features = output_size)

    def forward(self, x):
        # initialize hidden state
        #(numlayers. N, H_out)
        h0 = torch.zeros(self.layer_size,x.shape[0], self.hidden_size).requires_grad_()
        # Forward pass
        out, hn = self.rnn(x, h0.detach())
        out = out[:, -1, :] 
        out = self.fc(out)
        return out
        

## GRU

In [19]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, layer_size, output_size, dropout_prob):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size 
        self.layer_size = layer_size 

        # GRU Layer
        # (N, L, H_in), (num_layer, N, H_out) ---> (N, L, H_out), (num_layer, N, H_out)
        self.gru = nn.GRU(input_size = input_size, hidden_size = hidden_size, num_layers = layer_size, batch_first = True, dropout = dropout_prob)
        # FCC
        # (*, H_in) ---> (*, H_out)
        self.fc = nn.Linear(in_features=hidden_size, out_features = 1)
        
    def forward(self, x):
        # initialize hidden state
        #(numlayers. N, H_out)
        h0 = torch.zeros(self.layer_size,x.shape[0], self.hidden_size).requires_grad_()
        # Forward pass
        out, hn = self.gru(x, h0.detach())
        out = out[:, -1, :] 
        out = self.fc(out)
        return out


## LSTM

In [20]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, layer_size, output_size, dropout_prob):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size 
        self.layer_size = layer_size 

        # LSTM Layer
        # (N, L, H_in), (num_layer, N, H_out), (num_layer, N, H_cell) ---> (N, L, H_out), (num_layer, N, H_out), (num_layer, N, H_cell)
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = layer_size, batch_first = True, dropout = dropout_prob)
        # FCC
        # (*, H_in) ---> (*, H_out)
        self.fc = nn.Linear(in_features=hidden_size, out_features = 1)
        
    def forward(self, x):
        # initialize hidden state
        #(numlayers, N, H_out)
        h0 = torch.zeros(self.layer_size,x.shape[0], self.hidden_size).requires_grad_()
        c0 = torch.zeros(self.layer_size,x.shape[0], self.hidden_size).requires_grad_()
        # Forward pass
        out, (hn,cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = out[:, -1, :] 
        out = self.fc(out)
        return out

In [21]:
def get_model(model,  model_params):
    models = {
        "rnn" : RNNModel,
        "gru" : GRUModel,
        "lstm" : LSTMModel
    }
    return models.get(model.lower())(**model_params)

## Training

In [22]:
# Training
class Optimizer():
    def __init__(self, model, criterion, optimizer):
        self.model = model 
        self.criterion = criterion 
        self.optimizer = optimizer
        self.training_losses = []
        self.val_losses = []

    def train_step(self, X, y):
        # set the model to train mode
        self.model.train() # allows the weights of the network to be updated
        # Make predictions
        yhat = self.model.forward(X)
        # Compute loss
        loss = self.criterion(y, yhat)
        # Compute gradients
        loss.backward()
        self.optimizer.step()
        self.optimizer.zero_grad()
        # Return losses
        return loss.item()

    def train(self, train_loader, val_loader, batch_size = 64, n_epochs = 50, n_features = 1):
        for epoch in range(1, n_epochs + 1):
            batch_losses = []
            for X_batch, y_batch in train_loader:
                X_batch = X_batch.view([batch_size,  -1, n_features]) # add a dimension at 2, (64, 106) --> (64,106,1)
                loss = self.train_step(X_batch,  y_batch) # Calls the training step herem which calls model.forward()
                batch_losses.append(loss)
            training_loss = np.mean(batch_losses)
            self.training_losses.append(training_loss)

            with torch.no_grad():
                batch_val_losses = []
                for x_val, y_val in val_loader:
                    x_val = x_val.view([batch_size, -1, n_features]) # adds a dimension ar 2, (64, 106) --> 
                    self.model.eval()
                    yhat = model.forward(x_val) # Calls model.forward() method here
                    val_loss = self.criterion(y_val, yhat)
                    batch_val_losses.append(val_loss.item())
                validation_loss = np.mean(batch_val_losses)
                self.val_losses.append(validation_loss)

            if (epoch <= 10) | (epoch%50 == 0):
                print(f"[{epoch}/{n_epochs}] Training loss : {training_loss:.4f}\t Validation loss : {validation_loss:.4f}")

        torch.save(self.model.state_dict(),f"state_dict/example_model_{datetime.now()}")

    def evaluate(self, test_loader, batch_size = 1, n_features = 1):
        with torch.no_grad():
            predictions = []
            values = []
            for x_test, y_test in test_loader:
                x_test = x_test.view([batch_size, -1, n_features]) # (1, 106) --> (1, 106, 6)
                self.model.eval()
                yhat = self.model(x_test)
                predictions.append(yhat.detach().numpy())
                values.append(y_test.detach().numpy())

        return predictions, values

    def evaluate_train(self, train_loader, batch_size = 64, n_features = 1):
        with torch.no_grad():
            predictions = []
            values = []
            for x_train, y_train in train_loader:
                x_train = x_train.view([batch_size, -1, n_features])
                self.model.eval()
                yhat = self.model.forward(x_train)
                predictions.append(yhat.detach().numpy())
                values.append(y_train.detach().numpy())

        return predictions, values

## Inverse Transform

In [23]:
def inverse_transform(scaler,df, columns):
    for col in columns:
        df[col] = scaler.inverse_transform(df[col])
    return df



## Prediction

In [24]:
def format_predictions(predictions, values, df_test, scaler):
    vals = np.concatenate(values, axis = 0).ravel()
    preds = np.concatenate(predictions, axis = 0).ravel()
    df_result = pd.DataFrame(data = {"value" : vals, "predictions" : preds}, index = df_test.head(len(vals)).index)
    df_result = inverse_transform(scaler, df_result, [["value", "predictions"]])
    return df_result

## Metrics

In [25]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def calculate_metrics(df):
    return {'mae' : mean_absolute_error(df.value, df.predictions),
            'rmse' : mean_squared_error(df.value, df.predictions) ** 0.5,
            'r2' : r2_score(df.value, df.predictions)}

# --------------------------------------------------------------------------------------------------

## RNN

In [26]:
input_size = len(X_train.columns) # 106
output_size = 1
hidden_size = 64
layer_size = 1
batch_size = 64
dropout_prob = 0.2 
n_epochs = 500
learning_rate = 1e-3
weight_decay = 1e-6

model_params = {
    "input_size" : input_size,
    "hidden_size" : hidden_size,
    "layer_size" : layer_size, 
    "output_size" : output_size,
    "dropout_prob" : dropout_prob
}

model = get_model("rnn", model_params)
criterion = nn.MSELoss(reduction = "mean")
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay=weight_decay)
opt = Optimizer(model = model, criterion = criterion, optimizer = optimizer)
opt.train(train_loader=train_loader, val_loader=val_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_size)

[1/500] Training loss : 0.0363	 Validation loss : 0.0207
[2/500] Training loss : 0.0138	 Validation loss : 0.0122
[3/500] Training loss : 0.0099	 Validation loss : 0.0108
[4/500] Training loss : 0.0098	 Validation loss : 0.0210
[5/500] Training loss : 0.0107	 Validation loss : 0.0517
[6/500] Training loss : 0.0292	 Validation loss : 0.0129
[7/500] Training loss : 0.0290	 Validation loss : 0.0123
[8/500] Training loss : 0.0195	 Validation loss : 0.0184
[9/500] Training loss : 0.0161	 Validation loss : 0.0314
[10/500] Training loss : 0.0171	 Validation loss : 0.0231
[50/500] Training loss : 0.0141	 Validation loss : 0.0227
[100/500] Training loss : 0.0090	 Validation loss : 0.0169
[150/500] Training loss : 0.0060	 Validation loss : 0.0072
[200/500] Training loss : 0.0052	 Validation loss : 0.0152
[250/500] Training loss : 0.0027	 Validation loss : 0.0092
[300/500] Training loss : 0.0024	 Validation loss : 0.0088
[350/500] Training loss : 0.0117	 Validation loss : 0.0080
[400/500] Trainin

In [27]:
predictions, values = opt.evaluate(test_loader, batch_size = 64, n_features = input_size)

In [28]:
training_losses = opt.training_losses
validation_losses = opt.val_losses

p1 = go.Figure()
p1.add_trace(go.Scatter(y = training_losses, name = "training losses"))
p1.add_trace(go.Scatter(y = validation_losses, name = "validation losses"))
p1.update_layout(xaxis_title = "epochs", yaxis_title = "losses")

In [29]:


df_result = format_predictions(predictions, values, X_test, normaliser)
df_result

,value,predictions
Datetime,,
2022-03-26 04:00:00,519.0,557.382629
2022-03-26 05:00:00,533.0,563.345093
2022-03-26 06:00:00,575.0,565.120972
2022-03-26 07:00:00,581.0,583.599670
2022-03-26 08:00:00,538.0,590.276917
...,...,...
2022-04-08 07:00:00,717.0,790.632690
2022-04-08 08:00:00,718.0,780.526672
2022-04-08 09:00:00,748.0,779.341614


In [30]:
result_metrics = calculate_metrics(df_result)
result_metrics

{'mae': 35.417347, 'rmse': 44.10261499519119, 'r2': 0.8572788652361514}

In [31]:
p2 = go.Figure()
p2.add_trace(go.Scatter(x = df_result.index, y = df_result.value))
p2.add_trace(go.Scatter(x = df_result.index, y = df_result.predictions))

## GRU Prediction

In [32]:
input_size = len(X_train.columns) # 106
output_size = 1
hidden_size = 64
layer_size = 1
batch_size = 64
dropout_prob = 0.2 
n_epochs = 500
learning_rate = 1e-3
weight_decay = 1e-6

model_params = {
    "input_size" : input_size,
    "hidden_size" : hidden_size,
    "layer_size" : layer_size, 
    "output_size" : output_size,
    "dropout_prob" : dropout_prob
}

model = get_model("gru", model_params)
criterion = nn.MSELoss(reduction = "mean")
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay=weight_decay)
opt = Optimizer(model = model, criterion = criterion, optimizer = optimizer)
opt.train(train_loader=train_loader, val_loader=val_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_size)

/home/imantha/anaconda3/envs/ml/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1



[1/500] Training loss : 0.0512	 Validation loss : 0.0254
[2/500] Training loss : 0.0161	 Validation loss : 0.0138
[3/500] Training loss : 0.0102	 Validation loss : 0.0152
[4/500] Training loss : 0.0088	 Validation loss : 0.0173
[5/500] Training loss : 0.0092	 Validation loss : 0.0142
[6/500] Training loss : 0.0090	 Validation loss : 0.0138
[7/500] Training loss : 0.0108	 Validation loss : 0.0141
[8/500] Training loss : 0.0130	 Validation loss : 0.0118
[9/500] Training loss : 0.0146	 Validation loss : 0.0262
[10/500] Training loss : 0.0214	 Validation loss : 0.0332
[50/500] Training loss : 0.0051	 Validation loss : 0.0120
[100/500] Training loss : 0.0040	 Validation loss : 0.0065
[150/500] Training loss : 0.0030	 Validation loss : 0.0071
[200/500] Training loss : 0.0026	 Validation loss : 0.0076
[250/500] Training loss : 0.0025	 Validation loss : 0.0146
[300/500] Training loss : 0.0022	 Validation loss : 0.0090
[350/500] Training loss : 0.0020	 Validation loss : 0.0174
[400/500] Trainin

In [33]:
training_losses = opt.training_losses
validation_losses = opt.val_losses

p1 = go.Figure()
p1.add_trace(go.Scatter(y = training_losses, name = "training losses"))
p1.add_trace(go.Scatter(y = validation_losses, name = "validation losses"))
p1.update_layout(xaxis_title = "epochs", yaxis_title = "losses")

In [34]:
predictions, values = opt.evaluate(test_loader, batch_size = 64, n_features = input_size)

In [35]:
df_result = format_predictions(predictions, values, X_test, normaliser)
df_result

,value,predictions
Datetime,,
2022-03-26 04:00:00,519.0,543.101318
2022-03-26 05:00:00,533.0,547.302612
2022-03-26 06:00:00,575.0,548.252441
2022-03-26 07:00:00,581.0,568.505493
2022-03-26 08:00:00,538.0,571.489014
...,...,...
2022-04-08 07:00:00,717.0,805.309021
2022-04-08 08:00:00,718.0,796.790527
2022-04-08 09:00:00,748.0,797.343384


In [36]:
result_metrics = calculate_metrics(df_result)
result_metrics

{'mae': 35.94148, 'rmse': 47.9420095458761, 'r2': 0.8313477689995366}

In [37]:
p2 = go.Figure()
p2.add_trace(go.Scatter(x = df_result.index, y = df_result.value))
p2.add_trace(go.Scatter(x = df_result.index, y = df_result.predictions))

## LSTM Prediction

In [38]:
input_size = len(X_train.columns) # 106
output_size = 1
hidden_size = 64
layer_size = 2
batch_size = 64
dropout_prob = 0.2 
n_epochs = 500
learning_rate = 1e-3
weight_decay = 1e-6
model_params = {
    "input_size" : input_size,
    "hidden_size" : hidden_size,
    "layer_size" : layer_size, 
    "output_size" : output_size,
    "dropout_prob" : dropout_prob
}

model = get_model("lstm", model_params)
criterion = nn.MSELoss(reduction = "mean")
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay=weight_decay)
opt = Optimizer(model = model, criterion = criterion, optimizer = optimizer)
opt.train(train_loader=train_loader, val_loader=val_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_size)

[1/500] Training loss : 0.1063	 Validation loss : 0.0479
[2/500] Training loss : 0.0257	 Validation loss : 0.0195
[3/500] Training loss : 0.0123	 Validation loss : 0.0147
[4/500] Training loss : 0.0100	 Validation loss : 0.0129
[5/500] Training loss : 0.0085	 Validation loss : 0.0110
[6/500] Training loss : 0.0085	 Validation loss : 0.0094
[7/500] Training loss : 0.0104	 Validation loss : 0.0103
[8/500] Training loss : 0.0107	 Validation loss : 0.0084
[9/500] Training loss : 0.0117	 Validation loss : 0.0164
[10/500] Training loss : 0.0120	 Validation loss : 0.0217
[50/500] Training loss : 0.0055	 Validation loss : 0.0081
[100/500] Training loss : 0.0039	 Validation loss : 0.0078
[150/500] Training loss : 0.0035	 Validation loss : 0.0148
[200/500] Training loss : 0.0030	 Validation loss : 0.0106
[250/500] Training loss : 0.0030	 Validation loss : 0.0134
[300/500] Training loss : 0.0023	 Validation loss : 0.0092
[350/500] Training loss : 0.0021	 Validation loss : 0.0118
[400/500] Trainin

In [52]:
training_losses = opt.training_losses
validation_losses = opt.val_losses

p1 = go.Figure()
p1.add_trace(go.Scatter(y = training_losses, name = "training losses"))
p1.add_trace(go.Scatter(y = validation_losses, name = "validation losses"))
p1.update_layout(xaxis_title = "epochs", yaxis_title = "losses")
p1.update_layout(title = "LSTM - Mean Squred error on normalised data")

In [40]:
predictions, values = opt.evaluate(test_loader, batch_size = 64, n_features = input_size)

In [41]:
df_result = format_predictions(predictions, values, X_test, normaliser)
df_result

,value,predictions
Datetime,,
2022-03-26 04:00:00,519.0,534.951294
2022-03-26 05:00:00,533.0,541.091370
2022-03-26 06:00:00,575.0,543.586487
2022-03-26 07:00:00,581.0,564.907593
2022-03-26 08:00:00,538.0,581.699341
...,...,...
2022-04-08 07:00:00,717.0,808.920715
2022-04-08 08:00:00,718.0,799.465393
2022-04-08 09:00:00,748.0,798.204834


In [42]:
result_metrics = calculate_metrics(df_result)
result_metrics

{'mae': 35.97406, 'rmse': 49.388002639178474, 'r2': 0.8210208045865478}

In [55]:
p2 = go.Figure()
p2.add_trace(go.Scatter(x = df_result.index, y = df_result.value, name = "original ts"))
p2.add_trace(go.Scatter(x = df_result.index, y = df_result.predictions, name = "lstm prediction"))
p2.update_layout(title = "Energy Load")

### Prediction on training set

In [44]:
predictions, values = opt.evaluate_train(train_loader, batch_size = 64, n_features = input_size)

In [45]:
df_result = format_predictions(predictions, values, X_train, normaliser)
df_result

,value,predictions
Datetime,,
2021-11-16 19:00:00,545.401611,518.233826
2021-11-16 20:00:00,488.106293,511.068176
2021-11-16 21:00:00,494.472412,494.252289
2021-11-16 22:00:00,485.377930,491.472717
2021-11-16 23:00:00,480.830688,488.645416
...,...,...
2022-03-11 06:00:00,788.224426,809.158875
2022-03-11 07:00:00,765.488159,803.986633
2022-03-11 08:00:00,775.492126,798.272095


In [46]:
result_metrics = calculate_metrics(df_result)
result_metrics

{'mae': 16.510607, 'rmse': 21.073092460056216, 'r2': 0.963291827107398}

In [47]:
p2 = go.Figure()
p2.add_trace(go.Scatter(x = df_result.index, y = df_result.value))
p2.add_trace(go.Scatter(x = df_result.index, y = df_result.predictions))

In [48]:
df_result["delta"] = df_result["value"] - df_result["predictions"]
df_result

,value,predictions,delta
Datetime,,,
2021-11-16 19:00:00,545.401611,518.233826,27.167786
2021-11-16 20:00:00,488.106293,511.068176,-22.961884
2021-11-16 21:00:00,494.472412,494.252289,0.220123
2021-11-16 22:00:00,485.377930,491.472717,-6.094788
2021-11-16 23:00:00,480.830688,488.645416,-7.814728
...,...,...,...
2022-03-11 06:00:00,788.224426,809.158875,-20.934448
2022-03-11 07:00:00,765.488159,803.986633,-38.498474
2022-03-11 08:00:00,775.492126,798.272095,-22.779968


In [49]:
px.line(y = df_result["delta"])